<a href="https://colab.research.google.com/github/wnyngj/class2022spring/blob/main/huggingface_gradio0518%2C0523.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

### [Image classification](https://huggingface.co/tasks/image-classification)

해당 모델이 어떤 기능을 하는지는 숙지됐다고 가정하고 시험 문제 나옴

e.g. https://huggingface.co/google/vit-base-patch16-224 \

해당 그림이 1000개의 class들 중 어느그룹에 가까운지 classify하고 확률로 나타내는 model

How to use 있는지 꼬옥 확인!!하고 
copy paste한 것

In [ ]:
!pip install transformers #how to use엔 없지만 transformers를 깔아야gkwl

In [3]:
#[script형태]

from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg' \
#kitty 두마리 사진을 url로 불러옴 
image = Image.open(requests.get(url, stream=True).raw)
#img변수에 받음

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
#pretained 잘 학습이 된 행렬 = AI
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
#classify해내는 모델 변수

inputs = feature_extractor(images=image, return_tensors="pt")
#사진의 특징화할 수 있는 숫자 행렬 정보들만 뽑아옴
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes 1000개의 class 있음
predicted_class_idx = logits.argmax(-1).item()
#그 중 확률이 가장 큰 것 1개?
print("Predicted class:", model.config.id2label[predicted_class_idx])
#model.config.id2label[predicted_class_idx= 출력 class 이름

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


Demo in Gradio

그라디오로 모델 구현하기

이게 중요해요~~

In [6]:
#[함수형태]

def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
#아까 img변수를 input으로 받아옴

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
#아까 "어떻게 출력될 것인가" 부분을 변수로 받아내기
  return predicted_class
#출력  predicted_class

In [5]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")
#밑에 나올 예제 그림들 수업 사이트에 있는 url로 불러온 것

0

In [ ]:
#그라디로 형태로 만들어야지~

gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#(아까 정의한 함수, 인풋 유형 밝히고-이미지 , 아웃풋 유형 밝히기-텍스트) 

#examples = 그라디오 예시를 보고자 할 때 예시 몇가지가 있으면 파악하기 쉬워지니까
#예제 = 앞서 url로 불러온 사진 두 장

#ex)천개의 class들 중 하나를 불러옴, 출력 seashore, coast, seacoast

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

fill mask 설명 확인하기

text processing

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

#결과 리스트로 1,2,3,4,5개 딕셔너리가 있음

Demo in Gradio

In [11]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  #딕셔너리 형태이므로 데이터프레임화 하면 돼
  return df

In [13]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Token classification](https://huggingface.co/tasks/token-classification)

nlp중 하나의 분야

nlp understanding:nl가 주어져있을 때,컴퓨터는 이 대화에서 골라내기 해야함

어떤 명사는 정보적으로 더 중요함. 그것을 recognition하는 것

장소LOC, 사람PER, 물건ORG(삼성), Miscellaneous(MISC)

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#각각의 token들이 무엇인지 분류함

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
#nlp라는 모델
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
#ner_results: 윗문장을 입력으로 받았을 때의 출력
print(ner_results)
#두 개의 딕셔너리

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


Demo in Gradio

In [18]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df
  #리스트로 딕셔너리 있을 떄 바로 데이터프레임화 가능!

In [19]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

주어진 문장들 사이 유사성 
two texts - silmilarity

# 첫 input을 중간에 받아주는 것이 꼭 필요한데, 이는 vectors, embeddings, feature라고 부름

이렇게 받아주고 나서 vector들끼리 비교해서 유사성을 파악하는 방식임


e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)
#embeddings가 뭐야?
#[첫번째 문장[],두번째 문장[]]

In [ ]:
len(embeddings[1])
#384개의 숫자가 각각 문장을 embedding화 한 것임 = feature

두개의 벡터(embeddings) 있음
1. [1,2]
2. [2,1]
두 벡터를 좌표평면 위에 나타냄
이 두 벡터간 유사성은 각도(theta)로 파악할 수 있음

cos(theta=0)=1 유사도 100퍼센트!

cos(theta=90)=0 유사도 0퍼센트!

정리: 두 벡터간 유사성은 벡터 사이 각도값의 cos(theta), cosine similarity로 파악하면 된다.
if 벡터 3개라면 - 3차원에서

위의 예시처럼 차원이 384차원이더라도, 두벡터는 원점과 함께 삼각형의 관계를 이룰수밖에 없으므로 각도는 항상 나옴.




In [23]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
#embeddings[0]: 첫번째 문장을 이루는 384개의 숫자
#embeddings[1]: 두번째 문장을 이루는 384개의 숫자

#두벡터가 이루는 값의 각도의 cos값
cosine_scores

tensor([[0.5726]])

Demo in Gradio

In [24]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2]) #리스트화 한 후 
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores
  #cos값이므로 -1~1사이 값으로 나오고, 0에 가까울수록 similar하지 않은 것

In [ ]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()